get dataset

In [ ]:
# !wget https://github.com/alexeygrigorev/large-datasets/releases/download/dogs-cats/train.zip

In [1]:
import shutil
import glob

import os
import re

In [9]:
shutil.unpack_archive('train.zip', '')

* Create a `train` and `validation` folders
* In each folder, create `cats` and `dogs` folders

In [10]:
!mkdir train/cats
!mkdir train/dogs

!mkdir validation
!mkdir validation/cats
!mkdir validation/dogs

In [11]:
val_cats_folder = 'validation/cats'
train_cats_folder = 'train/cats'

val_dogs_folder = 'validation/dogs'
train_dogs_folder = 'train/dogs'

In [12]:
all_files = glob.glob('train/*.jpg')

* Move the first 10,000 images to the train folder (from 0 to 9999) for boths cats and dogs - and put them in respective folders
* Move the remaining 2,500 images to the validation folder (from 10000 to 12499)

In [13]:
for f in all_files:
    if 'cat' in f:
        _number_cat = int(re.findall(r'\d+', f)[0])
        if _number_cat < 10000:
            shutil.move(f, train_cats_folder)
        else:
            shutil.move(f, val_cats_folder)
        
    elif 'dog' in f:
        _number_dog = int(re.findall(r'\d+', f)[0])
        if _number_dog < 10000:
            shutil.move(f, train_dogs_folder)
        else:
            shutil.move(f, val_dogs_folder)

### Question 1

Since we have a binary classification problem, what is the best loss function for us?

* mean squared error
* categorical cross-entropy
* binary cross-entropy

### Question 2

What's the total number of parameters of the model? You can use the summary method for that.